In [1]:
#%pip install pettingzoo[atari]
#%pip install "autorom[accept-rom-license]"
#%pip install --find-links dist/ --no-cache-dir AutoROM[accept-rom-license]
#!pip install supersuit
#!pip install stable_baselines3
#!pip install --upgrade "supersuit>=3.5.0"


In [1]:
# Start AutoROM

#!AutoROM

# Import libraries
from pettingzoo.atari import warlords_v3
from pettingzoo.utils import BaseParallelWrapper
import gymnasium as gym
import numpy as np
from collections import defaultdict, Counter
import importlib
import os
import imageio
import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecMonitor, VecFrameStack
from pettingzoo.utils.conversions import aec_to_parallel
import time

In [2]:
import numpy as np
import supersuit as ss
from pettingzoo.atari import warlords_v3
from stable_baselines3.common.vec_env import VecMonitor, VecTransposeImage

def make_env(num_envs=4):
    # 1. Init environment
    env = warlords_v3.env(render_mode="rgb_array")

    # 2. Convert to parallel (for multi-agent)
    env = aec_to_parallel(env)

    # 3. Add wrappers
    env = ss.black_death_v3(env)  # Mask dead agents
    env = ss.frame_skip_v0(env, 4)  # Frame skipping (default in Atari)
    env = ss.dtype_v0(env, np.float32)  # Convert to float32
    env = ss.normalize_obs_v0(env, env_min=0, env_max=255)  # Normalize pixel values

    # 4. Convert to vectorized environment for SB3
    env = ss.pettingzoo_env_to_vec_env_v1(env)

    # 5. Run multiple copies in parallel (same agent logic reused)
    env = ss.concat_vec_envs_v1(env, num_vec_envs=num_envs, num_cpus=1, base_class="stable_baselines3")

    # 6. Reorder image axes to channel-first [C,H,W], needed for MlpPolicy
    #env = ss.flatten_v0(env)

    # 7. Optional: Monitor for logging
    env = VecMonitor(env)

    return env

env = make_env(num_envs=2)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=1_000_000, progress_bar=True)
model.save("warlords_parallel_model.zip")


/root/miniconda3/envs/pettingzooenv/lib/python3.11/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:78: UserWarning: The `render_mode` attribute is not defined in your environment. It will be set to None.
  warnings.warn("The `render_mode` attribute is not defined in your environment. It will be set to None.")


Using cuda device


Output()

/root/miniconda3/envs/pettingzooenv/lib/python3.11/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.57e+03 |
|    ep_rew_mean     | -0.5     |
| time/              |          |
|    fps             | 775      |
|    iterations      | 1        |
|    time_elapsed    | 21       |
|    total_timesteps | 16384    |
---------------------------------


...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.61e+03    |
|    ep_rew_mean          | -0.5        |
| time/                   |             |
|    fps                  | 283         |
|    iterations           | 58          |
|    time_elapsed         | 3355        |
|    total_timesteps      | 950272      |
| train/                  |             |
|    approx_kl            | 0.010062953 |
|    clip_fraction        | 0.0961      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.53       |
|    explained_variance   | 0.0774      |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0356     |
|    n_updates            | 570         |
|    policy_gradient_loss | -0.00187    |
|    value_loss           | 0.00362     |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.54e+03    |
|    ep_rew_mean          | -0.5        |
| time/                   |             |
|    fps                  | 283         |
|    iterations           | 59          |
|    time_elapsed         | 3411        |
|    total_timesteps      | 966656      |
| train/                  |             |
|    approx_kl            | 0.009024071 |
|    clip_fraction        | 0.0557      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.52       |
|    explained_variance   | 0.0415      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0342      |
|    n_updates            | 580         |
|    policy_gradient_loss | -0.0015     |
|    value_loss           | 0.00567     |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.59e+03    |
|    ep_rew_mean          | -0.5        |
| time/                   |             |
|    fps                  | 283         |
|    iterations           | 60          |
|    time_elapsed         | 3471        |
|    total_timesteps      | 983040      |
| train/                  |             |
|    approx_kl            | 0.009725094 |
|    clip_fraction        | 0.083       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.48       |
|    explained_variance   | 0.0465      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0133      |
|    n_updates            | 590         |
|    policy_gradient_loss | -0.0023     |
|    value_loss           | 0.00507     |
-----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.6e+03      |
|    ep_rew_mean          | -0.5         |
| time/                   |              |
|    fps                  | 283          |
|    iterations           | 61           |
|    time_elapsed         | 3521         |
|    total_timesteps      | 999424       |
| train/                  |              |
|    approx_kl            | 0.0136748785 |
|    clip_fraction        | 0.0929       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.52        |
|    explained_variance   | 0.0971       |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0184      |
|    n_updates            | 600          |
|    policy_gradient_loss | -0.00227     |
|    value_loss           | 0.00455      |
------------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.61e+03    |
|    ep_rew_mean          | -0.5        |
| time/                   |             |
|    fps                  | 283         |
|    iterations           | 62          |
|    time_elapsed         | 3578        |
|    total_timesteps      | 1015808     |
| train/                  |             |
|    approx_kl            | 0.008480958 |
|    clip_fraction        | 0.0454      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.5        |
|    explained_variance   | 0.0972      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00843     |
|    n_updates            | 610         |
|    policy_gradient_loss | -0.000421   |
|    value_loss           | 0.00513     |
-----------------------------------------
